### Goal: Calculate pi as fast as possible
 - Note: to start, we'll just use the normal arctan series for a challenge:
 $$ \frac{\pi}{4} = \sum_{k \ge 0} \frac{(-1)^k}{2k+1} $$
 - Since we have that $$ \frac{(-1)^{2k}}{2(2k)+1} + \frac{(-1)^{2k+1}}{2(2k+1)+1} = \frac{1}{4k+1} + \frac{-1}{4k+3} = \frac{2}{(4k+1)(4k+3)}$$ we can sum these terms two at a time to gain a bit of speed.

In [ ]:
from math import pi as real_pi
from tqdm import tqdm

def pi_python():
    total = 0
    # Add in reverse order to avoid floating point precision errors
    for k in tqdm(reversed(range(200000000)), total=200000000):
        total += 8. / ((4*k+1)*(4*k+3))
    return total

pi = pi_python()
print(abs(pi - real_pi))


"""
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000000/200000000 [05:06<00:00, 651779.52it/s]
2.5000002068509275e-09
"""

##### Compiling with `pypy`

Using the same script and running with pypy, we get much faster performance

```
$ pypy3 temp.py 
100%|███████████████████████| 200000000/200000000 [00:11<00:00, 16931114.41it/s]
2.5000002068509275e-09
$ pypy3 temp.py 
100%|██████████████████████| 2000000000/2000000000 [07:35<00:00, 4391991.59it/s]
2.5000002068509275e-10
```

##### `numpy`
Using numpy arrays to do the sum may be faster than using pure Python

In [21]:
import numpy as np

def pi_numpy():
    print("Constructing array...")
    arr = np.arange(200000000, dtype="int64")
    print("Calculating sum values...")
    vals = 8. / np.polynomial.polynomial.polyval(arr, [3,16,16])
    print("Sorting values (for stable sum)...")
    vals.sort()
    print("Summing values...")
    return vals.sum()

%time pi_numpy()

Constructing array...
Calculating sum values...
Sorting values (for stable sum)...
Summing values...
CPU times: user 9.34 s, sys: 1.19 s, total: 10.5 s
Wall time: 11 s


3.141592651089793

##### `numba`
Pulling out the big guns, let's just use `numba` to go fast.
For some reason I can't use numba in this Jupyter notebook...

In [28]:
import numba

ImportError: Numba needs NumPy 1.20 or less

Here's the script that I ran locally:
```py
from math import pi as real_pi
from numba import njit
from time import time

@njit
def pi_numba():
    total = 0
    for k in range(2000000000, -1, -1):
        total += 8. / ((4*k+1)*(4*k+3))
    return total

# Compile the function
pi_numba()

start = time()
pi = pi_numba()
end = time()
print(f"{end-start} seconds")
print(abs(pi - real_pi))
```

And here's some performance results:

num of terms = 2*10^8:
```
0.26631736755371094 seconds
2.5000002068509275e-09
```

num of terms = 2*10^9:
```
2.839431047439575 seconds
1.9004664508770475e-10
```

num of terms = 2*10^10:

```
27.300660371780396 seconds
1.6484650533499234e-08 (think there's some floating point error here)
```

##### More Precision - Using `gmpy2`
To get more digits, we use the library `gmpy2` for extended precision and use [this algorithm](https://en.wikipedia.org/wiki/Borwein%27s_algorithm#Quadratic_convergence_(1984)) that converges quadratically (i.e. each iteration roughly doubles the number of correct digits) to get 10 million digits

In [36]:
from gmpy2 import mpfr as D, sqrt, get_context
# Program to calculate 10 million digits of pi (takes ~15 minutes to run on my machine)
def pi_ten_million():
    get_context().precision = 100000000
    a = sqrt(2)
    b = 0
    p = 2+sqrt(2)
    for n in range(25):
        sqrt_a = sqrt(a)
        a_new = (sqrt_a + 1/sqrt_a)/2
        b_new = (1+b)*sqrt_a / (a+b)
        p_new = (1+a_new)*p*b_new / (1+b_new)
        a,b,p = a_new, b_new, p_new
        s = str(p)
        print(n,f"{s[:50]}...{s[-50:]}")
    f=open("pi.out","w")
    f.write(s[2:10000002])
    f.close()
%time pi_ten_million()

0 3.142606753941622600790719823618301891971356246277...41885525157660628919783367468821222935341984954532
1 3.141592660966044230497752235120339690679284256864...10626955767434753621664228587437969486786065467423
2 3.141592653589793238645773991757141794034789623867...84973197785151056460234474596637435819095048396754
3 3.141592653589793238462643383279502884197224120466...24826717979178074007880788421258562531503676994041
4 3.141592653589793238462643383279502884197169399375...06650593576457487229576208300565363642846109954489
5 3.141592653589793238462643383279502884197169399375...54590142318553169522447767053928600001935416567977
6 3.141592653589793238462643383279502884197169399375...61680336541960041075745570049286681719171065863476
7 3.141592653589793238462643383279502884197169399375...70716629211693529914432060007515388472206502039748
8 3.141592653589793238462643383279502884197169399375...34556829888212816730372632721171785648500631324219
9 3.141592653589793238462643383279502884197169